In [1]:
import re
import requests
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
import string
import warnings
warnings.filterwarnings("ignore")
# warnings.filterwarnings(action='once')
import matplotlib.pyplot as plt
%matplotlib inline
import category_encoders as ce
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

import lightgbm as lgb

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

seed = 42

In [2]:
def scale(scaler, data):
  if (scaler == 'standard'):
    std_scaler = StandardScaler()
    return (std_scaler.fit_transform(data))
  elif (scaler == 'minmax'):
    minmax_scaler = MinMaxScaler(feature_range=(-1,1))
    return (minmax_scaler.fit_transform(data))

# Load datasets

In [25]:
with open ('../data/fight_data_final.pkl', 'rb') as pklfile:
    fight_data_final = pickle.load(pklfile)

with open ('../data/fight_card_final.pkl', 'rb') as pklfile:
    fight_card_final = pickle.load(pklfile)

df_fight_card = pd.read_excel('../data/fight_card.xlsx', sheet_name='Sheet1')    

# Predict

## Shallow Learning

In [26]:
# Prepare train & test datasets

fight_data_final.drop(columns=['fighter1','fighter2', 'winner'], inplace=True)

X = scale('standard', fight_data_final.iloc[:,:-1])
y = fight_data_final.iloc[:,-1]

X_test = scale('standard', fight_card_final)

In [27]:
X.shape, y.shape, X_test.shape

((5305, 52), (5305,), (11, 52))

In [28]:
# LightGBM
import lightgbm as lgb

lgb_params = {
                 'n_estimators' : 500, 'boosting_type' : 'dart'
                 }

lgbm = lgb.LGBMClassifier(**lgb_params)
lgbm.fit(X, y)

y_pred_lgbm = lgbm.predict(X_test)
preds = []

for idx, val in enumerate(y_pred_lgbm):
    if (val == 0):
        preds.append(df_fight_card.fighter1[idx])
    else:
        preds.append(df_fight_card.fighter2[idx])

df_fight_card['Winner Prediction'] = preds

df_fight_card


,weight_class,fighter1,fighter2,Winner Prediction
0,Lightweight,Khabib Nurmagomedov,Dustin Poirier,Khabib Nurmagomedov
1,Lightweight,Edson Barboza,Paul Felder,Edson Barboza
2,Lightweight,Islam Makhachev,Davi Ramos,Islam Makhachev
3,Heavyweight,Curtis Blaydes,Shamil Abdurakhimov,Shamil Abdurakhimov
4,Lightweight,Mairbek Taisumov,Diego Ferreira,Diego Ferreira
5,Women's Flyweight,Joanne Calderwood,Andrea Lee,Andrea Lee
6,Women's Bantamweight,Liana Jojua,Sarah Moras,Sarah Moras
7,Lightweight,Ottman Azaitar,Teemu Packalen,Teemu Packalen
8,Welterweight,Belal Muhammad,Takashi Sato,Belal Muhammad
9,Welterweight,Nordine Taleb,Muslim Salikhov,Nordine Taleb


In [30]:
df_fight_card.to_csv('../data/fight_card.csv')

In [20]:
df_fight_card

,weight_class,fighter1,fighter2,Winner Prediction
0,Lightweight,Khabib Nurmagomedov,Dustin Poirier,Khabib Nurmagomedov
1,Lightweight,Edson Barboza,Paul Felder,Edson Barboza
2,Lightweight,Islam Makhachev,Davi Ramos,Islam Makhachev
3,Heavyweight,Curtis Blaydes,Shamil Abdurakhimov,Shamil Abdurakhimov
4,Lightweight,Mairbek Taisumov,Diego Ferreira,Diego Ferreira
5,Women's Flyweight,Joanne Calderwood,Andrea Lee,Andrea Lee
6,Women's Bantamweight,Liana Jojua,Sarah Moras,Sarah Moras
7,Lightweight,Ottman Azaitar,Teemu Packalen,Teemu Packalen
8,Welterweight,Belal Muhammad,Takashi Sato,Belal Muhammad
9,Welterweight,Nordine Taleb,Muslim Salikhov,Nordine Taleb
